In [10]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

glm_api_key = os.getenv("GLM_API_KEY")
glm_base_url = os.getenv("GLM_BASE_URL")

client = OpenAI(api_key=glm_api_key,base_url=glm_base_url)

In [11]:
model_name='glm-4-flash'

In [12]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flight_number",
            "description": "根据始发地、目的地和日期，查询对应日期的航班号",
            "parameters": {
                "type": "object",
                "properties": {
                    "departure": {
                        "description": "出发地",
                        "type": "string"
                    },
                    "destination": {
                        "description": "目的地",
                        "type": "string"
                    },
                    "date": {
                        "description": "日期",
                        "type": "string",
                    }
                },
                "required": [ "departure", "destination", "date" ]
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "查询某航班在某日的票价",
            "parameters": {
                "type": "object",
                "properties": {
                    "flight_number": {
                        "description": "航班号",
                        "type": "string"
                    },
                    "date": {
                        "description": "日期",
                        "type": "string",
                    }
                },
                "required": [ "flight_number", "date"]
            },
        }
    },
]

In [13]:
messages = []
messages.append({"role": "user", "content": "帮我查询从2024年1月20日，从北京出发前往上海的航班"})
response = client.chat.completions.create(
    model=model_name,  # 填写需要调用的模型名称
    messages=messages,
    tools=tools,
)
print(response.choices[0].message)
messages.append(response.choices[0].message.model_dump())

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_-9054952246455463902', function=Function(arguments='{"date": "2024-01-20", "departure": "北京", "destination": "上海"}', name='get_flight_number'), type='function', index=0)])


In [ ]:
messages = []
messages.append({"role": "system", "content": "不要假设或猜测传入函数的参数值。如果用户的描述不明确，请要求用户提供必要信息"})
# messages.append({"role": "user", "content": "帮我查询2024年1月20日1234航班的票价"})
messages.append({"role": "user", "content": "查询2024年1月20日1234航班"}) #也能查询到
response = client.chat.completions.create(
    model="glm-4-flash",  # 填写需要调用的模型名称
    messages=messages,
    tools=tools,
)
print(response.choices[0].message)
messages.append(response.choices[0].message.model_dump())

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_-9054950940784996043', function=Function(arguments='{"date": "2024-01-20", "flight_number": "1234"}', name='get_ticket_price'), type='function', index=0)])


In [15]:
def get_flight_number(date:str , departure:str , destination:str):
    flight_number = {
        "北京":{
            "上海" : "1234",
            "广州" : "8321",
        },
        "上海":{
            "北京" : "1233",
            "广州" : "8123",
        }
    }
    return { "flight_number":flight_number[departure][destination] }
def get_ticket_price(date:str , flight_number:str):
    return {"ticket_price": "1000"}

In [16]:
import json


def parse_function_call(model_response,messages):
    # 处理函数调用结果，根据模型返回参数，调用对应的函数。
    # 调用函数返回结果后构造tool message，再次调用模型，将函数结果输入模型
    # 模型会将函数调用结果以自然语言格式返回给用户。
    if model_response.choices[0].message.tool_calls:
        tool_call = model_response.choices[0].message.tool_calls[0]
        args = tool_call.function.arguments
        function_result = {}
        if tool_call.function.name == "get_flight_number":
            function_result = get_flight_number(**json.loads(args))
        if tool_call.function.name == "get_ticket_price":
            function_result = get_ticket_price(**json.loads(args))
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id":tool_call.id
        })
        response = client.chat.completions.create(
            model="glm-4-flash",  # 填写需要调用的模型名称
            messages=messages,
            tools=tools,
        )
        print(response.choices[0].message)
        messages.append(response.choices[0].message.model_dump())

In [34]:
# 清空对话
messages = []
 
messages.append({"role": "system", "content": "不要假设或猜测传入函数的参数值。如果用户的描述不明确，请要求用户提供必要信息"})
messages.append({"role": "user", "content": "帮我查询1月23日，北京到广州的航班"})
 
response = client.chat.completions.create(
    model="glm-4-flash",  # 填写需要调用的模型名称
    messages=messages,
    tools=tools,
)
print(response.choices[0].message)
messages.append(response.choices[0].message.model_dump())
 
parse_function_call(response,messages)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_-9054951250022731477', function=Function(arguments='{"date": "2022-01-23", "departure": "北京", "destination": "广州"}', name='get_flight_number'), type='function', index=0)])
ChatCompletionMessage(content='根据您的查询，我已经为您找到了1月23日，从北京到广州的航班号，航班号为8321。', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [37]:
messages

[{'role': 'system', 'content': '不要假设或猜测传入函数的参数值。如果用户的描述不明确，请要求用户提供必要信息'},
 {'role': 'user', 'content': '帮我查询1月23日，北京到广州的航班'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_-9054951250022731477',
    'function': {'arguments': '{"date": "2022-01-23", "departure": "北京", "destination": "广州"}',
     'name': 'get_flight_number'},
    'type': 'function',
    'index': 0}]},
 {'role': 'tool',
  'content': '{"flight_number": "8321"}',
  'tool_call_id': 'call_-9054951250022731477'},
 {'content': '根据您的查询，我已经为您找到了1月23日，从北京到广州的航班号，航班号为8321。',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None},
 {'role': 'user', 'content': '这趟航班的票价是多少？'},
 {'content': '请问您需要查询哪个航班的票价呢？',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None}]

In [36]:
# messages.append({"role": "user", "content": "这趟航班的价格是多少？"})
messages.append({"role": "user", "content": "这趟航班的票价是多少？"})
response = client.chat.completions.create(
    model="glm-4-flash",  # 填写需要调用的模型名称
    messages=messages,
    tools=tools,
)
print(response.choices[0].message)
messages.append(response.choices[0].message.model_dump())
 
parse_function_call(response,messages)

ChatCompletionMessage(content='请问您需要查询哪个航班的票价呢？', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
